In [44]:
import cv2
import random
import numpy as np
import copy
random.seed(72)

In [45]:
photos = cv2.imread('2-image.jpg')
print(photos.shape)

(360, 640, 3)


In [46]:
clusters = []
for i in range(3):
    x, y = random.randrange(photos.shape[0]), random.randrange(photos.shape[1])
    ci = photos[x,y,:]
    clusters.append([x,y])
print(clusters)

[[37, 608], [95, 352], [357, 557]]


In [47]:
def cal_k_mean(clusters):

    clusters_pts = []
    for i in range(len(clusters)):
        clusters_pts.append([clusters[i]])

    for i in range(photos.shape[0]):
        for j in range(photos.shape[1]):
            min_distance = 1e+20
            c = 0
            for cluster_idx in range(len(clusters)):
                distance = photos[i,j] - photos[clusters[cluster_idx][0], clusters[cluster_idx][1]]
                distance = np.linalg.norm(distance, ord=2)
                if distance < min_distance :
                    min_distance = distance
                    c = cluster_idx
            clusters_pts[c].append([i,j])
    return clusters_pts

In [48]:
def cal_mean(clusters):
    new_clusters = []
    for i in range(len(clusters)):
        x, y = np.floor(np.mean(clusters_pts[i], axis=0))
        new_clusters.append([x,y])
    return new_clusters

In [60]:
repeat = True
clusters_pts = cal_k_mean(clusters)
new_clusters = cal_mean(clusters_pts)
for i in range(len(new_clusters)):
    new_clusters[i][0], new_clusters[i][1] = int(new_clusters[i][0]), int(new_clusters[i][1])
print(new_clusters)



while(repeat):
    repeat = False
    ori = copy.deepcopy(new_clusters)
    clusters_pts = cal_k_mean(new_clusters)
    new_clusters = cal_mean(clusters_pts)
    print('new:',new_clusters)
    for j in range(len(new_clusters)):
        if ((new_clusters[j][0] - ori[j][0])>1e-6 or (new_clusters[j][1] - ori[j][1])>1e-6):
            repeat = True
        new_clusters[j][0], new_clusters[j][1] = int(new_clusters[j][0]), int(new_clusters[j][1])
    print(new_clusters)


[[61, 324], [194, 244], [227, 324]]
new: [[237.0, 315.0], [72.0, 329.0], [83.0, 317.0]]
[[237, 315], [72, 329], [83, 317]]
new: [[87.0, 324.0], [243.0, 315.0], [80.0, 329.0]]
[[87, 324], [243, 315], [80, 329]]
new: [[88.0, 339.0], [92.0, 324.0], [245.0, 315.0]]
[[88, 339], [92, 324], [245, 315]]
new: [[247.0, 315.0], [91.0, 338.0], [95.0, 324.0]]
[[247, 315], [91, 338], [95, 324]]
new: [[97.0, 323.0], [247.0, 315.0], [91.0, 338.0]]
[[97, 323], [247, 315], [91, 338]]
new: [[99.0, 340.0], [97.0, 323.0], [247.0, 315.0]]
[[99, 340], [97, 323], [247, 315]]
new: [[90.0, 336.0], [247.0, 315.0], [97.0, 323.0]]
[[90, 336], [247, 315], [97, 323]]
new: [[247.0, 315.0], [97.0, 323.0], [98.0, 341.0]]
[[247, 315], [97, 323], [98, 341]]
new: [[97.0, 323.0], [247.0, 315.0], [89.0, 336.0]]
[[97, 323], [247, 315], [89, 336]]
new: [[98.0, 341.0], [97.0, 323.0], [247.0, 315.0]]
[[98, 341], [97, 323], [247, 315]]
new: [[90.0, 336.0], [247.0, 315.0], [97.0, 323.0]]
[[90, 336], [247, 315], [97, 323]]
new: [[

KeyboardInterrupt: 